In [1]:
import os
import cv2
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from tensorflow.keras.preprocessing import image
from tensorflow import keras
from tensorflow.keras.applications import resnet50, ResNet50
from tensorflow.keras.preprocessing import image


In [2]:
model = tf.keras.models.load_model(r'E:\AImodel\models\Multilabel\SideLunge-face-multilabel-model02.h5')
dir_path = r'E:\AImodel\models\Multilabel\test-images'

print(model.input.shape)
print(model.input.dtype)
print(model.output.shape)
print(model.output.dtype)


(None, 128, 128, 3)
<dtype: 'float32'>
(None, 5)
<dtype: 'float32'>


In [1]:
def preprocess_image(image_path, target_size=(128, 128)):
    """이미지 파일을 불러와 전처리하는 함수"""
    img = image.load_img(image_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # 모델 예측을 위해 차원 확장
    img_array /= 255.0  # 이미지를 0과 1 사이로 스케일링
    return img_array


def load_and_preprocess_from_directory(directory_path, target_size=(128, 128)):
    """지정된 디렉토리 내의 모든 이미지를 불러와 전처리하는 함수"""
    processed_images = []
    for file_name in os.listdir(directory_path):
        file_path = os.path.join(directory_path, file_name)
        if file_path.lower().endswith(('.png', '.jpg', '.jpeg')):
            img = preprocess_image(file_path, target_size=target_size)
            processed_images.append(img)
    return np.vstack(processed_images)  # 전처리된 이미지들을 하나의 numpy 배열로 합침


def apply_sliding_window_to_predictions(predictions, window_size=5, method='mean'):
    """2D 예측 결과 배열에 대해 슬라이딩 윈도우를 적용하는 함수"""
    num_classes = predictions.shape[1]  # 클래스 수
    smoothed_predictions = np.zeros_like(predictions)
    
    for class_idx in range(num_classes):
        class_predictions = predictions[:, class_idx]  # 현재 클래스에 대한 모든 예측값
        # 각 예측값에 대해 슬라이딩 윈도우 적용
        for i in range(len(class_predictions)):
            start_idx = max(0, i - window_size // 2)
            end_idx = min(len(class_predictions), i + window_size // 2 + 1)
            window = class_predictions[start_idx:end_idx]
            if method == 'mean':
                smoothed_value = np.mean(window)
            elif method == 'median':
                smoothed_value = np.median(window)
            else:
                raise ValueError("Method should be 'mean' or 'median'")
            smoothed_predictions[i, class_idx] = smoothed_value
    
    return smoothed_predictions



def print_predictions(predictions, threshold=0.5):
    # 각 라벨마다 T, F의 개수를 저장할 딕셔너리 초기화
    label_counts = {
        1: {'T': 0, 'F': 0}, 
        2: {'T': 0, 'F': 0}, 
        3: {'T': 0, 'F': 0}, 
        4: {'T': 0, 'F': 0}, 
        5: {'T': 0, 'F': 0}
    }
    
    for i, prediction in enumerate(predictions):
        labels = (prediction > threshold).astype(int)
        label_str = ','.join(['T' if label == 1 else 'F' for label in labels])
        print(f"Image {i+1}: {label_str}")
        
        # 각 라벨마다 T, F의 개수를 계산
        for label_index, label in enumerate(labels, start=1):
            if label == 1:
                label_counts[label_index]['T'] += 1
            else:
                label_counts[label_index]['F'] += 1
    
    # 각 라벨에 대한 총합 출력
    for label_index in label_counts:
        print(f"Label {label_index}: T = {label_counts[label_index]['T']}, F = {label_counts[label_index]['F']}")




preprocessed_images = load_and_preprocess_from_directory(dir_path)
predictions = model.predict(preprocessed_images, verbose=0)
# 예측 결과 출력
print(predictions)
smoothed_predictions = apply_sliding_window_to_predictions(predictions, window_size=5, method='mean')




NameError: name 'dir_path' is not defined

In [4]:
# 모델의 출력층에 접근하여 클래스의 개수 얻기
output_layer = model.layers[-1]  # 모델의 마지막 층 (출력층)을 가져옴
num_classes = output_layer.units  # 출력층의 유닛 수를 가져옴, 이는 클래스의 수와 동일

print(f'모델은 총 {num_classes}개의 클래스를 가지고 있습니다.')
class_labels = []
# 클래스 라벨 정의 (예: num_classes 개의 클래스)
for i in range(num_classes):
    class_labels.append(f'class{i+1}')

# 예를 들어, num_classes가 3이라면 class_labels는 ['class1', 'cd lass2', 'class3']이 됩니다.


# predictions 배열에서 각 입력에 대한 모든 클래스의 확률 출력
for i, prediction in enumerate(predictions):
    print(f'Image {i+1}:')
    for j, class_label in enumerate(class_labels):
        print(f'  {class_label}: {prediction[j]:.2f}%')
    print()  # 각 이미지의 예측 결과 사이에 공백 추가


print_predictions(predictions, 0.5)



모델은 총 5개의 클래스를 가지고 있습니다.
Image 1:
  class1: 0.01%
  class2: 0.01%
  class3: 0.87%
  class4: 0.99%
  class5: 0.98%

Image 2:
  class1: 0.01%
  class2: 0.02%
  class3: 0.74%
  class4: 0.99%
  class5: 0.12%

Image 3:
  class1: 0.00%
  class2: 0.00%
  class3: 1.00%
  class4: 0.99%
  class5: 0.00%

Image 4:
  class1: 0.00%
  class2: 0.01%
  class3: 0.99%
  class4: 0.00%
  class5: 0.00%

Image 5:
  class1: 0.00%
  class2: 0.00%
  class3: 1.00%
  class4: 0.00%
  class5: 0.00%

Image 6:
  class1: 0.38%
  class2: 0.09%
  class3: 0.91%
  class4: 0.07%
  class5: 0.03%

Image 7:
  class1: 0.02%
  class2: 0.03%
  class3: 0.56%
  class4: 0.94%
  class5: 0.76%

Image 8:
  class1: 0.01%
  class2: 0.00%
  class3: 0.97%
  class4: 0.98%
  class5: 0.00%

Image 9:
  class1: 0.00%
  class2: 0.24%
  class3: 0.86%
  class4: 0.00%
  class5: 0.00%

Image 10:
  class1: 0.02%
  class2: 0.03%
  class3: 0.94%
  class4: 0.90%
  class5: 0.04%

Image 11:
  class1: 0.16%
  class2: 0.00%
  class3: 0.99%
  class4: 0.47%


In [5]:
print(f'모델은 총 {num_classes}개의 클래스를 가지고 있습니다.')
class_labels = []
# 클래스 라벨 정의 (예: num_classes 개의 클래스)
for i in range(num_classes):
    class_labels.append(f'class{i+1}')

# 예를 들어, num_classes가 3이라면 class_labels는 ['class1', 'cd lass2', 'class3']이 됩니다.


# predictions 배열에서 각 입력에 대한 모든 클래스의 확률 출력
for i, prediction in enumerate(smoothed_predictions):
    print(f'Image {i+1}:')
    for j, class_label in enumerate(class_labels):
        print(f'  {class_label}: {prediction[j]:.2f}%')
    print()  # 각 이미지의 예측 결과 사이에 공백 추가
# 부드러운 예측 결과 출력


print_predictions(smoothed_predictions, 0.5)

모델은 총 5개의 클래스를 가지고 있습니다.
Image 1:
  class1: 0.01%
  class2: 0.01%
  class3: 0.87%
  class4: 0.99%
  class5: 0.37%

Image 2:
  class1: 0.00%
  class2: 0.01%
  class3: 0.90%
  class4: 0.74%
  class5: 0.28%

Image 3:
  class1: 0.00%
  class2: 0.01%
  class3: 0.92%
  class4: 0.59%
  class5: 0.22%

Image 4:
  class1: 0.08%
  class2: 0.03%
  class3: 0.93%
  class4: 0.41%
  class5: 0.03%

Image 5:
  class1: 0.08%
  class2: 0.03%
  class3: 0.89%
  class4: 0.40%
  class5: 0.16%

Image 6:
  class1: 0.08%
  class2: 0.03%
  class3: 0.89%
  class4: 0.40%
  class5: 0.16%

Image 7:
  class1: 0.08%
  class2: 0.07%
  class3: 0.86%
  class4: 0.40%
  class5: 0.16%

Image 8:
  class1: 0.09%
  class2: 0.08%
  class3: 0.85%
  class4: 0.58%
  class5: 0.17%

Image 9:
  class1: 0.04%
  class2: 0.06%
  class3: 0.86%
  class4: 0.66%
  class5: 0.16%

Image 10:
  class1: 0.06%
  class2: 0.06%
  class3: 0.94%
  class4: 0.65%
  class5: 0.07%

Image 11:
  class1: 0.12%
  class2: 0.19%
  class3: 0.86%
  class4: 0.50%
